In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
project_path = '/content/drive/My Drive/Code Documentation Project/Issue Classification/'

In [ ]:
from numpy import loadtxt
import pandas as pd
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from sklearn import preprocessing


issue_df = pd.read_csv(project_path+'literature_comments_dataset.csv')


label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(issue_df['Code'])
issue_df['Code (Original)'] = issue_df['Code']
issue_df['Code'] = label_encoder.transform(issue_df['Code'])

#number_of_class
class_id_map_df = issue_df.drop_duplicates(subset=['Code'])
total_class=len(class_id_map_df)
encoded_label_list = class_id_map_df['Code'].to_list()
original_label_list = class_id_map_df['Code (Original)'].to_list()
class_id_map = dict(zip(encoded_label_list, original_label_list))

categ = ['aa','begauth','has_code','first_turn','last_turn']
le = preprocessing.LabelEncoder()
issue_df[categ] = issue_df[categ].apply(le.fit_transform)

print(len(issue_df))


In [ ]:
# build the lightgbm model
from sklearn.metrics import classification_report
import lightgbm as lgb


train_df=issue_df
X_train = train_df[['len','tloc','cloc','tpos1','tpos2','clen','tlen','ppau','npau','aa','begauth','has_code','first_turn','last_turn']] #'Full Length',
y_train = train_df['Code']

clf = lgb.LGBMClassifier(objective="multiclass", class_weight='balanced', num_classes=13, metric='multi_logloss', feature_pre_filter= False,
                             boosting_type="gbdt", lambda_l1=5.6172275504022865e-05, lambda_l2= 0.10617995541784066,feature_fraction= 0.7,
                             bagging_fraction= 1.0, bagging_freq= 0, min_child_samples= 20,num_leaves= 127)
    
clf.fit(X_train, y_train)

In [ ]:

test_df = pd.read_csv(project_path+'sentencewithstateruntime.csv')

categ = ['aa','begauth','has_code','first_turn','last_turn']
le = preprocessing.LabelEncoder()
test_df[categ] = test_df[categ].apply(le.fit_transform)

print(len(test_df))
#y_pred=clf.predict(X_test)
X_test = test_df[['len','tloc','cloc','tpos1','tpos2','clen','tlen','ppau','npau','aa','begauth','has_code','first_turn','last_turn']] #'Full Length',

y_pred=clf.predict(X_test)
label_list = []
for i in range(13):
  label_list.append(class_id_map[i])
pred_labels=[]
for e in y_pred:
  pred_labels.append(class_id_map[e])  

test_df['Code']=pred_labels

test_df.Code.value_counts().to_csv(project_path+'infotype_cnt.csv')